# 数理最適化とは

# 線形最適化問題

In [1]:
from gurobipy import *
model = Model("lo1")
x1 = model.addVar(name="x1")
x2 = model.addVar(name="x2")
x3 = model.addVar(ub=30, name="x3")
model.update()
model.addConstr(2*x1 + x2 + x3 <= 60)
model.addConstr(x1 + 2*x2 + x3 <= 60)
model.setObjective(15*x1 + 18*x2 + 30*x3, GRB.MAXIMIZE)
model.optimize()
print("Opt. Value", model.ObjVal)
for v in model.getVars():
    print(v.VarName, v.X)

Academic license - for non-commercial use only - expires 2021-06-06
Using license file /Users/yoshidayuto/gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x2b324efa
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e+01, 3e+01]
  Bounds range     [3e+01, 3e+01]
  RHS range        [6e+01, 6e+01]
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.3000000e+31   3.000000e+30   3.300000e+01      0s
       3    1.2300000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds
Optimal objective  1.230000000e+03
Opt. Value 1230.0
x1 10.0
x2 10.0
x3 30.0


# 整数最適化問題

In [2]:
from gurobipy import *
model = Model("puzzle")
x = model.addVar(vtype="I")
y = model.addVar(vtype="I")
z = model.addVar(vtype="I")
model.update()
model.addConstr(x + y + z == 32) 
model.addConstr(2*x + 4*y + 8*z ==80)
model.setObjective(y + z, GRB.MINIMIZE)
model.optimize()
print("Opt. Val.", model.ObjVal)
print("(x,y,z)=", x.X, y.X, z.X)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x9688a20c
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 8e+01]
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+00, best bound 4.000000000000e+00, gap 0.0000%
Opt. Val. 4.0
(x,y,z)= 28.0 2.0 2.0


## 輸送問題

In [3]:
# 問題例のデータ準備
d = {1:80, 2:270, 3:250, 4:160, 5:180}
M = {1:500, 2:500, 3:500}
I = [1, 2, 3, 4, 5]
J = [1, 2, 3]
c = {(1,1):4, (1,2):6, (1,3):9,
     (2,1):5, (2,2):4, (2,3):7,
     (3,1):6, (3,2):3, (3,3):4,
     (4,1):8, (4,2):5, (4,3):3,
     (5,1):10, (5,2):8, (5,3):4
    }

In [4]:
model = Model("transportation")
x = {}
for i in I:
    for j in J:
        x[i,j] = model.addVar(vtype="C", name="x({},{})".format(i,j))
model.update()

In [5]:
for i in I:
    model.addConstr(quicksum(x[i,j] for j in J)==d[i], name="Demand{}".format(i))

In [6]:
for j in J:
    model.addConstr(quicksum(x[i,j] for i in I) <= M[j], name="Capacity{}".format(j))

In [7]:
model.setObjective(quicksum(c[i,j]*x[i,j] for(i,j) in x), GRB.MINIMIZE)

In [8]:
model.optimize()
print("Opt. Val", model.ObjVal)
EPS = 1.e-6
for (i,j) in x:
    if x[i,j].X > EPS:
        print("sending quantity {} from factory {} to customer {}".format(x[i,j].X, j, i))

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 15 columns and 30 nonzeros
Model fingerprint: 0xa225ad1d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 5e+02]
Presolve time: 0.01s
Presolved: 8 rows, 15 columns, 30 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.3500000e+03   2.000000e+01   0.000000e+00      0s
       1    3.3700000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  3.370000000e+03
Opt. Val 3370.0
sending quantity 80.0 from factory 1 to customer 1
sending quantity 20.0 from factory 1 to customer 2
sending quantity 250.0 from factory 2 to customer 2
sending quantity 250.0 from factory 2 to customer 3
sending quantity 160.0 from factory 3 to customer 4
sending quant

# 双対問題

In [9]:
print("Const. Name: Slack, Dual")
for c in model.getConstrs():
    print("{}: {}, {}".format(c.ConstrName, c.slack, c.Pi))

Const. Name: Slack, Dual
Demand1: 0.0, 4.0
Demand2: 0.0, 5.0
Demand3: 0.0, 4.0
Demand4: 0.0, 3.0
Demand5: 0.0, 4.0
Capacity1: 400.0, 0.0
Capacity2: 0.0, -1.0
Capacity3: 160.0, 0.0


# 多品種輸送問題

In [10]:
produce = {1:[2, 4], 2:[1,2,3], 3:[2,3,4]}

In [11]:
d = {(1,1):80, (1,2):85, (1,3):300, (1,4):6,
     (2,1):270, (2,2):160, (2,3):400, (2,4):7,
     (3,1):250, (3,2):130, (3,3):350, (3,4):4,
     (4,1):160, (4,2):60, (4,3):200, (4,4):3,
     (5,1):180, (5,2):40, (5,3):150, (5,4):5
    }

In [12]:
I = set([i for (i,k) in d])

In [13]:
J,M = multidict({1:3000, 2:3000, 3:3000})

In [14]:
K,weight = multidict({1:5, 2:2, 3:3, 4:4})

In [15]:
cost = {(1,1):4, (1,2):6, (1,3):9,
        (2,1):5, (2,2):4, (2,3):7,
        (3,1):6, (3,2):3, (3,3):4,
        (4,1):8, (4,2):5, (4,3):3,
        (5,1):10, (5,2):8, (5,3):4
       }

In [16]:
c = {}
for i in I:
    for j in J:
        for k in produce[j]:
            c[i,j,k] = cost[i,j] * weight[k]

In [17]:
model = Model("multi-commondity transportation")
x = {}
for i,j,k in c:
    x[i,j,k] = model.addVar(vtype="C")
model.update()

In [18]:
for i in I:
    for k in K:
        model.addConstr(quicksum(x[i,j,k] for j in J if(i,j,k) in x) == d[i,k])

In [19]:
for j in J:
    model.addConstr(quicksum(x[i,j,k] for (i,j2,k) in x if j2==j) <= M[j])

In [20]:
model.setObjective(quicksum(c[i,j,k] * x[i,j,k] for (i,j,k) in x), GRB.MINIMIZE)

In [21]:
model.optimize()
print("Optimal value", model.ObjVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 23 rows, 40 columns and 80 nonzeros
Model fingerprint: 0x77b0a6c4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+03]
Presolve removed 23 rows and 40 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.3536000e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  4.353600000e+04
Optimal value 43536.0


# 混合問題

In [22]:
a = {(1,1):.25, (1,2):.15, (1,3):.3,
     (2,1):.3, (2,2):.15, (2,3):.3,
     (3,1):.15, (3,2):.65, (3,3):.05,
     (4,1):.1, (4,2):.05, (4,3):.85
    }
I,p = multidict({1:5, 2:6, 3:8, 4:20})
K,LB,UB = multidict({1:[.1,.2], 2:[.0,.35], 3:[.45,1.0]})
model = Model("product mix")
x = {}
for i in I:
    x[i] = model.addVar()
model.update()
model.addConstr(quicksum(x[i] for i in I) == 1)
for k in K:
    model.addConstr(quicksum(x[i]*a[i,k] for i in I) <= UB[k])
    model.addConstr(quicksum(x[i]*a[i,k] for i in I) >= LB[k])
model.setObjective(quicksum(x[i]*p[i] for i in I), GRB.MINIMIZE)
model.optimize()
for i in I:
    print(i, x[i].X)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 7 rows, 4 columns and 28 nonzeros
Model fingerprint: 0x6ee8e5ef
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+00]
Presolve removed 3 rows and 0 columns
Presolve time: 0.00s
Presolved: 4 rows, 6 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+00   9.482000e-01   0.000000e+00      0s
       2    9.6216216e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds
Optimal objective  9.621621622e+00
1 0.6486486486486487
2 0.0
3 0.05405405405405392
4 0.29729729729729737


# 分数最適化

In [23]:
model = Model("fractional 1")
x = model.addVar()
y = model.addVar()
z = model.addVar()
t = model.addVar()
model.update()
model.addConstr(x + y + z == 32*t)
model.addConstr(2*x + 4*y == 1)
model.addConstr(2*x + 4*y + 8*z == 80*t)
model.setObjective(x + y, GRB.MINIMIZE)
model.optimize()
print("Opt.Val.=", model.ObjVal, "t=", t.X)
print("(x,y,z)=", x.X/t.X, y.X/t.X, z.X/t.X)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3 rows, 4 columns and 10 nonzeros
Model fingerprint: 0x4c564055
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  4.000000000e-01
Opt.Val.= 0.4 t= 0.0125
(x,y,z)= 23.999999999999996 8.0 0.0


# 多制約0-1ナップサック問題

In [24]:
def example():
    J,v = multidict({1:16, 2:19, 3:23, 4:28})
    a = {(1,1):2, (1,2):3, (1,3):4, (1,4):5,
         (2,1):3000, (2,2):3500, (2,3):5100, (2,4):7200
        }
    I,b = multidict({1:7, 2:10000})
    return I,J,v,a,b

In [25]:
def mkp(I,J,v,a,b):
    model = Model("mkp")
    x = {}
    for j in J:
        x[j] = model.addVar(vtype="B", name="x{}".format(j))
    model.update()
    for i in I:
        model.addConstr(quicksum(x[j]*a[i,j] for j in J) <= b[i])
    model.setObjective(quicksum(v[j]*x[j] for j in J), GRB.MAXIMIZE)
    model.update()
    return model

In [26]:
I,J,v,a,b = example()
model = mkp(I,J,v,a,b)

In [27]:
model.update()
model.write("mkp.lp")

In [28]:
model.update()
model.write("mkp.mps")

In [29]:
model.optimize()
print("Optimal Value=", model.ObjVal)
EPS = 1.e-6
for v in model.getVars():
    if v.X > EPS:
        print(v.VarName, v.X)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0x56a7c1ff
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [2e+00, 7e+03]
  Objective range  [2e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 1e+04]
Found heuristic solution: objective 35.0000000
Presolve removed 2 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 42 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.200000000000e+01, best bound 4.200000000000e+01, gap 0.0000%
Optimal Value= 42.0
x2 1.0
x3 1.0


# 栄養問題

In [61]:
F,c,d = multidict({
    "CQPounder":[360, {"Cal":556, "Carbo":39, "Protein":30, "VitA":147, "VitC":10, "Calc":221, "Iron":2.4}],
    "Big M":[320, {"Cal":556, "Carbo":46, "Protein":26, "VitA":97, "VitC":9, "Calc":142, "Iron":2.4}],
    "FFilet":[270, {"Cal":356, "Carbo":42, "Protein":14, "VitA":28, "VitC":1, "Calc":76, "Iron":0.7}],
    "Chicken":[290, {"Cal":431, "Carbo":45, "Protein":20, "VitA":9, "VitC":2, "Calc":37, "Iron":0.9}],
    "Fries":[190, {"Cal":249, "Carbo":30, "Protein":3, "VitA":0, "VitC":5, "Calc":7, "Iron":0.6}],
    "Milk":[170, {"Cal":138, "Carbo":10, "Protein":7, "VitA":80, "VitC":2, "Calc":227, "Iron":0}],
    "VegJuice":[100, {"Cal":69, "Carbo":17, "Protein":1, "VitA":750, "VitC":2, "Calc":18, "Iron":0}]
})
N,a,b = multidict({
    "Cal":[2000, 3000],
    "Carbo":[300, 375],
    "Protein":[50, 60],
    "VitA":[500, 750],
    "VitC":[85, 100],
    "Calc":[660, 900],
    "Iron":[6.0, 7.5]
})

In [62]:
model = Model("modern diet")
x = {}
for j in F:
    x[j] = model.addVar(vtype="I", name="x({})".format(j))
model.update()
for i in N:
    model.addConstr(quicksum(x[j]*d[j][i] for j in F) >= a[i], name="NurtrLB{}".format(i))
    model.addConstr(quicksum(x[j]*d[j][i] for j in F) <= b[i], name="NurtrUB{}".format(i))
model.setObjective(quicksum(x[j]*c[j] for j in F), GRB.MINIMIZE)
model.optimize()
print(model.status)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 14 rows, 7 columns and 92 nonzeros
Model fingerprint: 0x6408f388
Variable types: 0 continuous, 7 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-01, 8e+02]
  Objective range  [1e+02, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 3e+03]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
3


In [63]:
model.computeIIS()
for c in model.getConstrs():
    if c.IISConstr:
        print(c.ConstrName)


Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0        14         -         0         7         -           0s
        5         5         -         3         3         -           0s

IIS computed: 5 constraints, 3 bounds
IIS runtime: 0.01 seconds
NurtrUBCal
NurtrUBCarbo
NurtrUBProtein
NurtrLBVitC
NurtrUBIron


In [64]:
model.feasRelaxS(1, True, False, True)
model.optimize()
status = model.Status
if status == GRB.Status.OPTIMAL:
    print("Opt. Value", model.ObjVal)
    for v in model.getVars():
        print(v.VarName, v.X)


Solve phase I feasrelax model to determine minimal relaxation

Optimize a model with 14 rows, 21 columns and 106 nonzeros
Model fingerprint: 0x59d23df7
Model has 14 quadratic objective terms
Variable types: 14 continuous, 7 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-01, 8e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 3e+03]
Found heuristic solution: objective 5.600000e+19
Presolve time: 0.00s
Presolved: 14 rows, 21 columns, 106 nonzeros
Presolved model has 14 quadratic objective terms
Variable types: 14 continuous, 7 integer (0 binary)

Root relaxation: objective 6.893077e+02, 18 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  689.30771    0    3 5.6000e+19  689.30771   100%     -    0s
H    0     0                    107577.02352  689